In [ ]:
val userhome = System.getProperty("user.home")
val path = s"${userhome}/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"

interp.load.cp(os.Path(path))

import $ivy.`org.plotly-scala:plotly-almond_2.13:0.8.2`
import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import plotly._, element._, layout._, Plotly._
import ujson.Value
import sys.process._


In [ ]:
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import better.files._

import scala.util.Random
import edu.colorado.plv.bounder.Driver
import edu.colorado.plv.bounder.Action
import edu.colorado.plv.bounder.RunConfig
import upickle.default.read
import upickle.default.write
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}
import edu.colorado.plv.bounder.ExperimentsDb
import edu.colorado.plv.bounder.BounderUtil
import scala.concurrent.duration._
import scala.language.postfixOps

In [ ]:
val db = new ExperimentsDb(Some(path))

In [ ]:
import slick.driver.H2Driver.api._
import slick.jdbc.GetResult
import slick.jdbc.SQLActionBuilder
import scala.concurrent.Await

In [ ]:
case class Count(n:Int)
implicit val getCountResult = GetResult(r => Count(r.<<))

Await.result(db.db.run(sql"""select count(*) from results;""".as[Count]), 30 seconds)

In [ ]:
val triageFolder=s"${userhome}/Desktop/ossExpDis"

In [ ]:
//Download witnessed with no limit
// db.downloadResults(File("/Users/shawnmeier/Desktop/2appExp"), """%Witnessed%""", None)
db.downloadResults(File(triageFolder), """%""", None)

In [ ]:
// remove other jobs from target cfg

import edu.colorado.plv.bounder.RunConfig
import edu.colorado.plv.bounder.ir.AppLoc
import edu.colorado.plv.bounder.symbolicexecutor.state.ReceiverNonNull

def specifyCFG(folder:File) = {
//     val folder = triageFolder / "com.gianlu"/"sample"/"res_3841"

    val oldCfg = folder / "job_config"

    val rCfg = read[RunConfig](oldCfg.contentAsString)

    val appLoc = read[AppLoc]((folder / "res_loc").contentAsString)
    val mname = appLoc.method.simpleName
    val mclazz = appLoc.method.classType
    val newConfig = rCfg.copy(initialQuery = rCfg.initialQuery.filter{
        case ReceiverNonNull(cn,mn,ln,_) => 
        cn == mclazz && mn == mname && appLoc.toString.contains(ln.toString)
        case _ => false
    })
    val newCFG = folder / "cfg.json"
    newCFG.overwrite(write(newConfig))
}

// specifyCFG(triageFolder / "org.andstatus" /"sample"/"res_4028")
val baseOut = File("/Users/shawnmeier/Desktop/55select/")
val appList = List(
"com.duckduckgo",
"com.gianlu",
"com.health",
"com.lyonbros",
"org.andstatus",
)

baseOut.glob("**/res_*").filter(_.isDirectory).foreach{d => 
    try{
        specifyCFG(d)
    }catch{
        case e => println(s"failed to extract $d")
    }
}
// specifyCFG(File("/Users/shawnmeier/Desktop/55select/com.lyonbros/sample/res_266"))

In [ ]:
read[ruhnFile("/Users/shawnmeier/Desktop/55select/com.lyonbros/sample/res_266/job_config")

In [ ]:
val path = File("/Users/shawnmeier/Desktop/55select/org.andstatus/sample/res_4045/out/org.andstatus/sample")

Driver.readDB(path)

In [ ]:
def extractWitness(base:File) = {
    val dataFile = (base / "data.zip").unzip()
    val pathsF = dataFile.glob("**/paths.db")
    pathsF.next.moveToDirectory(base)
    Driver.readDB(base)
}
val path = s"${triageFolder}/com.gianlu/sample/data_477"
// val path = "/Users/shawnmeier/Desktop/55select/com.gianlu/sample/res_3841/out/com.gianlu/sample"
extractWitness(File(path))

In [ ]:
// Find locations with no successors
Driver.readDB(File(path),true)